# Programming Task
## Mehdi Hosseini Moghadam

##Task 3

In [2]:
pip install lime shap

In [3]:
!git clone https://github.com/RGU-Computing/DisCERN-XAI.git

Cloning into 'DisCERN-XAI'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 85 (delta 33), reused 50 (delta 17), pack-reused 0
Unpacking objects: 100% (85/85), done.


In [4]:
cd /content/DisCERN-XAI

/content/DisCERN-XAI


# ChatBot for Managing Conversation 

In [5]:
!pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 3.6 MB/s 
     |████████████████████████████████| 2.9 MB 11.2 MB/s 
     |████████████████████████████████| 3.3 MB 41.9 MB/s 
     |████████████████████████████████| 1.2 MB 36.4 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 636 kB 49.2 MB/s 
     |████████████████████████████████| 895 kB 60.2 MB/s 
     |████████████████████████████████| 109 kB 57.6 MB/s 
     |████████████████████████████████| 546 kB 51.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=c7de348fa5a3537f11856bf2762c88f0bb11a3168dc86b7481c6a16aeeae12f9
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd
import torch
embedder = SentenceTransformer('distilbert-multilingual-nli-stsb-quora-ranking')

Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.81k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/589 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
df = pd.read_excel("/content/Bot data.xlsx")
df


Greet  ...                     Hi how can I help you?
0   Greet  ...                     Hi how can I help you?
1   Greet  ...                     Hi how can I help you?
2   Greet  ...                     Hi how can I help you?
3   Greet  ...                     Hi how can I help you?
4   Greet  ...                     Hi how can I help you?
5   Greet  ...                     Hi how can I help you?
6     Bye  ...                       Bye, have a nice day
7     Bye  ...                       Bye, have a nice day
8     Bye  ...                       Bye, have a nice day
9    Help  ...  OK, Please provide us with following data
10   Help  ...  OK, Please provide us with following data
11   Help  ...  OK, Please provide us with following data
12   Help  ...  OK, Please provide us with following data

[13 rows x 3 columns]

In [8]:
def Train_Bot(path,name,column_id):
  display("Importing Data")

  df = pd.read_excel(path)
  row = df["{}".format(df.columns[column_id])].tolist()
  for i in range(len(row)):
     row[i] = str(row[i])

  display("Vectorizing Data")

  corpus =  row
  corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True,show_progress_bar=True,batch_size=16)
  torch.save(corpus_embeddings,"{}.pt".format(name))
  
  return("Training is Done.")


In [9]:
Train_Bot("/content/Bot data.xlsx","Bot",1)

'Importing Data'

'Vectorizing Data'

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


'Training is Done.'

In [14]:
def Search_query(excel_path, model_name, question_column_id, q, top_k,answers_column_id):  

    df = pd.read_excel(excel_path)
    row = df["{}".format(df.columns[question_column_id])].tolist()
    for i in range(len(row)):
     row[i] = str(row[i])


    c = 0
    l = []
    queries = q
    corpus =  row
    top_k = top_k


    corpus_embeddings = torch.load("/content/DisCERN-XAI/{}.pt".format(model_name))
    for query in queries:
        query_embedding = embedder.encode(query, convert_to_tensor=True)
        cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
        cos_scores = cos_scores.cpu()

        #We use np.argpartition, to only partially sort the top_k results
        top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

        # print("\n\n======================\n\n")
        # print("Query:", query)
        for idx in top_results[0:top_k]:
            l = l + [{"id": "{}".format(c),
                      "intent": df.iloc[int(idx)%df.shape[0],0], 
                        "answer":df.iloc[int(idx)%df.shape[0],answers_column_id],
                        "score":float(cos_scores[idx]),
                        "question":corpus[idx].strip()
            }]
            c +=1
            # print(corpus[idx].strip(),"\n", "(Score: %.4f)" % (cos_scores[idx]), "\n",df.iloc[int(idx)%df.shape[0],1],"\n+++++++++++++++++++++++\n")
    d = pd.DataFrame(l)
    dd = d.drop_duplicates(subset ="answer",
                     keep = "first")
    return(dd)        


# Scrip to Gather Data from User and send it to XAI model

In [15]:
import numpy as np
def data_collection():
    keys = ['Age', 'Gender', 'Alcohol use', 'Dust Allergy', 'chronic Lung Disease',
          'Smoking', 'Chest Pain', 'Fatigue', 'Shortness of Breath', 'Wheezing',
          'Swallowing Difficulty', 'Cough', 'Level','Desiered Level']
    format_dic = {'Age': "an integer format",
                'Gender': {2: 0},
                'Alcohol use': {2: 0},
                'Dust Allergy': {2: 0},
                  'chronic Lung Disease' : {2: 0},
                'Smoking': {2: 0},
                'Chest Pain': {2: 0},
                'Fatigue': {2: 0},
                'Shortness of Breath': {2: 0},
                'Wheezing': {2: 0},
                  'Swallowing Difficulty': {2: 0},
                  'Cough': {2: 0},
                  'Level': "{'Low','Medium','High'}",
                  'Desiered Level': "{'Low','Medium','High'}"}       
    data = {key: None for key in keys}

    for i in range(len(data)):
      key = input("Bot: \n Please Enter your {} with below info:\n {} \n".format(keys[i],format_dic[keys[i]]))
      if keys[i] == 'Level' or 'Desiered Level' :
        data[keys[i]] = str(key)
      else:  
        data[keys[i]] = int(key)
    q = np.asarray(list(data.values())[:-2],dtype="float32")
    label = str(list(data.values())[-2])
    d_label = str(list(data.values())[-1])    
    return(q, label, d_label)

In [16]:
from discern.discern_tabular import DisCERNTabular
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def test_cancer_risk(q,label,d_label):
    data_df = pd.read_csv('/content/DisCERN-XAI/tests/lung_cancer.csv')
    data_df = data_df.replace({'Level': {'Low': 0, 'Medium': 1, 'High': 2}})
    data_df = data_df.replace({'Gender': {2: 0}})
    data_df = data_df.replace({'Alcohol use': {2: 0}})
    data_df = data_df.replace({'Dust Allergy': {2: 0}})
    data_df = data_df.replace({'Smoking': {2: 0}})
    data_df = data_df.replace({'Chest Pain': {2: 0}})
    data_df = data_df.replace({'Fatigue': {2: 0}})
    data_df = data_df.replace({'Shortness of Breath': {2: 0}})
    data_df = data_df.replace({'Wheezing': {2: 0}})
    data_df = data_df.replace({'Swallowing Difficulty': {2: 0}})
    data_df = data_df.replace({'Cough': {2: 0}})
    data_df = data_df.replace({'chronic Lung Disease': {2: 0}})
    # print("Reading data complete!",data_df.columns)

    df = data_df.copy()
    x = df.loc[:, df.columns != 'Level'].values
    y = df['Level'].values
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=1)
    scaler = MinMaxScaler()
    x_train= scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    rfx = RandomForestClassifier(n_estimators=500)
    rfx.fit(x_train, y_train)
    x_test = q
    cat_indices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

    sparsity = []
    proximity = []
    discern = DisCERNTabular(rfx, 'LIME', 'Q')
    discern.init_data(x_train, y_train, [c for c in df.columns if c!='Level'], ['Low', 'Medium', 'High'], cat_feature_indices=cat_indices)
    cf, s, p = discern.find_cf(x_test, {'Low': 0, 'Medium': 1, 'High': 2}[label], desired_class=d_label)
    sparsity.append(s)
    proximity.append(p)
    _sparsity = sum(sparsity)/len(sparsity)
    _proximity = sum(proximity)/(len(proximity)*_sparsity)
    for i in range(len(data_df.columns)-1):
      if cf[i] == x_test[i]:
        pass
      else:
        print("--------------------------------------------------------------------------------")
        print("you need to change {} from {} to {}".format(data_df.columns[i],x_test[i],cf[i]))  
        print("--------------------------------------------------------------------------------")

In [19]:
flag = True
while flag:
  q = str(input("User Message: "))
  responce = Search_query("/content/Bot data.xlsx","Bot",1,[q],1,2)
  if responce["intent"][0] == "Help":
    print("Bot: ",str(responce["answer"].to_list()[0])) 
    user_data = data_collection()
    test_cancer_risk(user_data[0],user_data[1],user_data[2])
  else:
    print("Bot: ",str(responce["answer"].to_list()[0]))  
    

User Message: Hi


`max_length` is ignored when `padding`=`True`.


Bot:  Hi how can I help you?
User Message: AI model said my income is below 50K. I need help to increase my income over 50K
Bot:  OK, Please provide us with following data
Bot: 
 Please Enter your Age with below info:
 an integer format 
18
Bot: 
 Please Enter your Gender with below info:
 {2: 0} 
1
Bot: 
 Please Enter your Alcohol use with below info:
 {2: 0} 
1
Bot: 
 Please Enter your Dust Allergy with below info:
 {2: 0} 
0
Bot: 
 Please Enter your chronic Lung Disease with below info:
 {2: 0} 
1
Bot: 
 Please Enter your Smoking with below info:
 {2: 0} 
1
Bot: 
 Please Enter your Chest Pain with below info:
 {2: 0} 
1
Bot: 
 Please Enter your Fatigue with below info:
 {2: 0} 
0
Bot: 
 Please Enter your Shortness of Breath with below info:
 {2: 0} 
1
Bot: 
 Please Enter your Wheezing with below info:
 {2: 0} 
1
Bot: 
 Please Enter your Swallowing Difficulty with below info:
 {2: 0} 
1
Bot: 
 Please Enter your Cough with below info:
 {2: 0} 
0
Bot: 
 Please Enter your Level with bel

`max_length` is ignored when `padding`=`True`.


Bot:  Bye, have a nice day


KeyboardInterrupt: ignored